In [1]:
import pandas as pd

In [2]:
%%time
df_method_1 = pd.read_excel(fr"C:\Users\User\Desktop\Final Year Project\Code\Processed_CX\04_Method_1_Raw_Data\CX_Processed_Method_1.xlsx")
# df_hko = pd.read_csv(r'C:\Users\User\Desktop\Final Year Project\Code\Processed_HKO\HKO_full_joined_data_with_tolerance.csv')
df_aware  = pd.read_excel(r'C:\Users\User\Desktop\Final Year Project\Code\Processed_HKO\HKO_aware_concatenated_data.xlsx')
df_lidar  = pd.read_excel(r'C:\Users\User\Desktop\Final Year Project\Code\Processed_HKO\HKO_lidar_concatenated_data.xlsx')
df_wtws = pd.read_excel(r'C:\Users\User\Desktop\Final Year Project\Code\Processed_HKO\HKO_wtws_concatenated_data.xlsx')
df_wp1 = pd.read_csv(r"C:\Users\User\Desktop\Final Year Project\Code\Processed_HKO\processed_wp\SHW_windprofiler_concatenated.csv")
df_wp2 = pd.read_csv(r"C:\Users\User\Desktop\Final Year Project\Code\Processed_HKO\processed_wp\SLW_windprofiler_concatenated.csv")

CPU times: total: 7min 6s
Wall time: 7min 6s


In [3]:
%%time
# Convert the time columns to datetime
df_method_1['adjusted landing time'] = pd.to_datetime(df_method_1['adjusted landing time'])
df_aware['DATETIME'] = pd.to_datetime(df_aware['DATETIME'])
df_lidar['DATETIME'] = pd.to_datetime(df_lidar['DATETIME'])
df_wtws['DATETIME'] = pd.to_datetime(df_wtws['DATETIME'])
df_wp1['datetime'] = pd.to_datetime(df_wp1['datetime'])
df_wp2['datetime'] = pd.to_datetime(df_wp2['datetime'])

CPU times: total: 438 ms
Wall time: 424 ms


In [4]:
placeholder_value = pd.Timestamp('1900-01-01')  # example placeholder value
df_method_1['adjusted landing time'].fillna(placeholder_value, inplace=True)

In [6]:
%%time
# Sort the DataFrames by their respective time columns
df_method_1 = df_method_1.sort_values('adjusted landing time')
df_aware = df_aware.sort_values('DATETIME')
df_lidar = df_lidar.sort_values('DATETIME')
df_wtws = df_wtws.sort_values('DATETIME')
df_wp1 = df_wp1.sort_values('datetime')
df_wp2 = df_wp2.sort_values('datetime')

CPU times: total: 844 ms
Wall time: 830 ms


In [8]:
%%time
# Define a dictionary with your dataframes and their corresponding prefixes
dfs = {'aware': df_aware, 'lidar': df_lidar, 'wtws': df_wtws}

df_merged = df_method_1.copy()

tolerance_level = pd.Timedelta(minutes=15)  

for prefix, df in dfs.items():
    # Add prefix to each column name including 'DATETIME', but only if it doesn't already exist
    df.columns = [f'{prefix}_{col}' if not col.startswith(f'{prefix}_') else col for col in df.columns]
    
    df[f'{prefix}_DATETIME'] = pd.to_datetime(df[f'{prefix}_DATETIME'])
    df = df.sort_values(f'{prefix}_DATETIME')
    
    df_merged = pd.merge_asof(left=df_merged, right=df,
                              left_on='adjusted landing time',
                              right_on=f'{prefix}_DATETIME',
                              tolerance=tolerance_level,
                              direction='backward')


CPU times: total: 1.09 s
Wall time: 1.08 s


In [11]:
df_merged.isna().sum()

Offset                                                   4984
UTC time                                               290964
Airframe                                                    0
Corrected Latitude (deg)                                    0
Corrected Longitude (deg)                                   0
Heading (true) (deg)                                    62840
Wind Direction (true) (deg)                              8011
Wind Speed (knots)                                       5976
Airspeed (calibrated; 1 or Only) (knots)                    0
Baro-Corrected Altitude (ft)                            63961
Radio Altitude (1, left, Capt or Only) (ft)             55393
Vertical Speed (inertial) (ft/min)                          0
Acceleration (normal load-factor) (g's)                     0
Angle of Attack (Indicated) (LH or only) (deg)         244972
Angle of Attack (Corrected) (LH or only) (deg)              0
Pitch Attitude (Capt or Only) (deg)                         0
Airport 

In [12]:
# AWS & WP
# Define a dictionary with your dataframes and their corresponding prefixes
dfs = {'windprofiler': df_wp1, 'windprofiler': df_wp2}

tolerance_level = pd.Timedelta(minutes=15)  

for prefix, df in dfs.items():
    # Add prefix to each column name including 'DATETIME', but only if it doesn't already exist
    df.columns = [f'{prefix}_{col}' if not col.startswith(f'{prefix}_') else col for col in df.columns]
    
    df_merged = pd.merge_asof(left=df_merged, right=df,
                              left_on='adjusted landing time',
                              right_on=f'{prefix}_datetime',
                              tolerance=tolerance_level,
                              direction='backward')


In [14]:
df_merged.isna().sum()

Offset                          4984
UTC time                      290964
Airframe                           0
Corrected Latitude (deg)           0
Corrected Longitude (deg)          0
                               ...  
windprofiler_SLW_snr_V_750     20459
windprofiler_SLW_snr_V_800     20459
windprofiler_SLW_snr_V_850     20459
windprofiler_SLW_snr_V_900     20459
windprofiler_SLW_snr_V_950     20459
Length: 350, dtype: int64

In [13]:
df_merged

,Offset,UTC time,Airframe,Corrected Latitude (deg),Corrected Longitude (deg),Heading (true) (deg),Wind Direction (true) (deg),Wind Speed (knots),Airspeed (calibrated; 1 or Only) (knots),Baro-Corrected Altitude (ft),...,windprofiler_SLW_snr_V_500,windprofiler_SLW_snr_V_550,windprofiler_SLW_snr_V_600,windprofiler_SLW_snr_V_650,windprofiler_SLW_snr_V_700,windprofiler_SLW_snr_V_750,windprofiler_SLW_snr_V_800,windprofiler_SLW_snr_V_850,windprofiler_SLW_snr_V_900,windprofiler_SLW_snr_V_950
0,NaN,5:27:03,88,22.382524,114.164636,251.455093,244.687500,11.000,175.750,4015.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10290,NaN,46,22.292823,113.842152,75.322273,142.031250,8.625,139.000,1056.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10291,13:59:32,46,22.293035,113.842823,74.443367,136.406250,7.125,141.000,1052.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10310,NaN,46,22.297385,113.856124,74.091805,106.171875,13.000,151.000,788.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10309,NaN,46,22.297170,113.855426,73.388680,92.812500,11.750,151.625,804.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754548,8013,NaN,113,22.326550,113.897753,NaN,87.187500,0.000,53.125,16.0,...,13.0,10.0,7.0,10.0,10.0,6.0,3.0,1.0,5.0,9.0
754549,8014,NaN,113,22.326446,113.897495,NaN,87.187500,0.000,50.500,24.0,...,13.0,10.0,7.0,10.0,10.0,6.0,3.0,1.0,5.0,9.0
754550,8015,8:9:22,113,22.326344,113.897278,241.786926,87.187500,0.000,48.500,28.0,...,13.0,10.0,7.0,10.0,10.0,6.0,3.0,1.0,5.0,9.0
754551,8016,NaN,113,22.326242,113.897061,NaN,87.187500,0.000,47.750,28.0,...,13.0,10.0,7.0,10.0,10.0,6.0,3.0,1.0,5.0,9.0


In [15]:
# Count the number of NaN values in each column
nan_counts = df_merged.isnull().sum()
print(nan_counts)

Offset                          4984
UTC time                      290964
Airframe                           0
Corrected Latitude (deg)           0
Corrected Longitude (deg)          0
                               ...  
windprofiler_SLW_snr_V_750     20459
windprofiler_SLW_snr_V_800     20459
windprofiler_SLW_snr_V_850     20459
windprofiler_SLW_snr_V_900     20459
windprofiler_SLW_snr_V_950     20459
Length: 350, dtype: int64


In [16]:
# Save the merged result back to a new CSV file, if needed
df_merged.to_csv(r'C:\Users\User\Desktop\Final Year Project\Code\Processed_Both\merged_CX_HKO_v3_1.csv', index=False)